In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from utils import *

In [2]:
img = Image.open('test.jpg')
array2 = np.asarray(img)

In [12]:
tflite_model = tf.lite.Interpreter(model_path='./model/newer/MobileNet.tflite')
tflite_model.allocate_tensors()

In [35]:
Sx = tflite_model.get_tensor_details()[31]['quantization_parameters']['scales']           .astype(float)
Sw_scale_list = tflite_model.get_tensor_details()[32]['quantization_parameters']['scales'].astype(float)
Sa = tflite_model.get_tensor_details()[33]['quantization_parameters']['scales']           .astype(float)

fraction_list = []
shift_list = []
M_list = []

for i in Sw_scale_list:
    M_list.append((Sx * i) / Sa)

for i in Sw_scale_list:
    fraction, shift = InterFrExp((Sx * i) / Sa)
    fraction_list.append(fraction)
    shift_list.append(shift)

for i in fraction_list:
    print(hex(i))

0x7e20edc0
0x480bb26e
0x6c3a3aaa
0x660c1cfa
0x46a02e36
0x5f2b095a
0x4868c69e
0x608293e5
0x4c0ea9d2
0x41af7024
0x5b41ae01
0x4d7d0b02
0x76c621a0
0x45cd3b42
0x7b19c6e0
0x7e809dcb
0x7285542a
0x4933d6ab
0x494f319f
0x7b013362
0x6ccc3559
0x5743411c
0x465580fc
0x532e4ee2
0x71521e2a
0x52f03b89
0x5e4e0167
0x77c038d4
0x51d77a67
0x73a21a0b
0x51f8ff88
0x4bb8d718


In [36]:
fraction_list = np.asarray(fraction_list)
fraction_list += 1

In [6]:
w = np.asarray(tflite_model.get_tensor(32))
b = np.asarray(tflite_model.get_tensor(29))

In [4]:
o = [[[0 for i in range(112)] for j in range(112)] for k in range(32)]
o = np.asarray(o)
o.astype(np.int64)

of = [[[0 for i in range(112)] for j in range(112)] for k in range(32)]
of = np.asarray(of)
of.astype(int)

print()

In [38]:
for n in range(32):
    for f_x in range(0, 224, 2):
        for f_y in range(0, 224, 2):
            local_sum = 0
            for c in range(3):
                for k_x in range(3):
                    for k_y in range(3):
                        if (f_x + k_x) != -1 and (f_y + k_y) != -1 and (f_x + k_x) < 224 and (f_y + k_y) < 224:
                            local_sum += w[n][k_x][k_y][c] * array2[f_x + k_x][f_y + k_y][c]
                        else:
                            local_sum += 0
            local_sum += b[n]
            local_sum = local_sum if local_sum > 0 else 0

            o[n][f_x >> 1][f_y >> 1] = (np.int64(local_sum) * fraction_list[n] >> 31) >> (abs(shift_list[n]))
            of[n][f_x >> 1][f_y >> 1] = int(local_sum * M_list[n])

In [40]:
print(o[0:32,0,0])

[31  0 21  0  1  0  0 26 12  0  0 12  0 13  0 10 19 10  4  3  3  6  0  0
 15  0  0  1  0 17 12 10]


In [34]:
print(of[0:32,0,1])

[39  0 18  0  0  0  0 29 22  0  0 21  0 24  0 18 29 19  0  0 13  0  0  0
 27  0  0 12  0 24  0 22]


In [3]:
from utils import *
o3 = model_invoke(array2)

In [25]:
print(o3[0:32, 1, 0])

[ 0  0  0  0 13  0  0  0 22  0 16  0  3  6  0 15  1 22  0  7 18  0  0 18
  0 13 13 15  0  0  0  0]


In [ ]:
pool_list = []

for c in range(1024):
    local_sum = 0
    for x in range(7):
        for y in range(7):
            local_sum += o3[c][x][y]
    local_sum = np.int64(local_sum) * 1530651995 >> 34
    pool_list.append(local_sum)

In [13]:
w, b, M, shift = get_param(tflite_model, 88, 30)

In [16]:
result = []
for i in range(43):
    local_sum = 0
    for j in range(1024):
        local_sum += w[i][j] * pool_list[j]
    local_sum += b[i]
    local_sum = (np.int64(local_sum) * M[0] >> 31) >> abs(shift[0])
    result.append(local_sum)

In [21]:
print(np.asarray(result)+124)

[ 73 106 158  94 116 110  51  68  98  79  58  54  47  47  92  51  65  34
  58  57  35  61   0  55  38  51  50  54  40  51  46  47   9  76  68  53
  65  58  93  48  82  49  23]
